# Bayesian amplitude regression
## Part 1:

In the lectures we have covered Bayesian networks.
Here we'll put this into practice by building a Bayesian neural network in pytorch to perform a regression task on the LHC amplitude for the production of two photons and a gluon in a gluon-gluon collision.

We want to train a network to predict the amplitude for the process $gg\rightarrow\gamma\gamma g$.  So the amplitude depends on the 4-momentum of 5 particles: 2 incoming gluons, 2 outgoing photons, and one outgoing gluon.  

The incoming gluons will have no transverse momentum, but their total momentum along the beam pipe is not necessarily zero.

The network we will train is a simple fully connected dense network.  This means that the input and output can only be vectors of real numbers.  We want to input the kinematic information on the particles to the network, and train the network to output the corresponding amplitude.

We will start by constructing a Bayesian layer in pytorch, and then building the Bayesian loss function.  We will then construct a Bayesian network from these layers, and use it to perform the amplitude regression. We will implement the variational inference and the repulsive ensemble versions of Bayesian neural networks.

This notebook is inspired by https://arxiv.org/abs/2412.12069

## Motivation

Amplitudes are phase-space dependent probabilities whihc are predicted in QFT.
Knowing the amplitude is one of the first steps needed to generate collider events in simulators. The first-principled prediction is completely deterministic, therefore there is no systematic noise from measurements in the quantity we are trying to regress.

We can exploit this aspect and introduce noise ourselves to test the ability of the neural networks to learn systematic errors, in a similar way to a closure test.

#### Outline / tasks:
 - Imports \& plotting set-up
 - Loading the data
 - Visualising the data
     - visualise some of the kinematics of the process (transverse momentum of photons/gluons, MET)
     - histogram the amplitudes
 - Preprocessing the data
     - neural networks like $\mathcal{O}(1)$ numbers
     - how should we preprocess the data?
 - Datasets and dataloaders
     - details are in the pytorch docs
 - Building a Bayesian layer
 - Constructing the Bayesian loss function
 - Building the Bayesian neural network
 - Optimising the neural network
 - Plot the train and validation losses as a function of the epochs
 - Visualize the predictions

     
Most practical pytorch skills you need for this work is covered in the basics tutorial at https://pytorch.org/tutorials/beginner/basics/intro.html.

Please download the training data `tutorial-2-data.zip` and extract it to the folder `data/tutorial-2-data/`.

If you want to run this tutorial in google colab, you can open a new colab and then upload this file.

The data can be downloaded using

In [ ]:
!wget -O tutorial-2-data.zip https://www.dropbox.com/s/n5e66w91rgmbqz2/dlpp-data.zip?dl=0&file_subpath=%2Fdlpp-data%2Ftutorial-2-data
!unzip "tutorial-2-data.zip"
!mkdir tutorial-2-data
!mv dlpp-data/tutorial-2-data/* tutorial-2-data/.
!rm -r __MACOSX/
!rm -r dlpp-data/
!ls

Make sure you switch to a GPU runtime to fully utilize the colab.

### Imports

In [ ]:
import os
import sys
import random
import time
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn import Module

#### Plotting set-up

In [ ]:
from matplotlib import pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties
import matplotlib.colors as mcolors
import colorsys


## Loading the data

In [ ]:
trn_dat = np.load("tutorial-2-data/trn_dat.npy")
trn_amp = np.load("tutorial-2-data/trn_amp.npy")

val_dat = np.load("tutorial-2-data/val_dat.npy")
val_amp = np.load("tutorial-2-data/val_amp.npy")

tst_dat = np.load("tutorial-2-data/tst_dat.npy")
tst_amp = np.load("tutorial-2-data/tst_amp.npy")

In [ ]:
print(f"train data shape: {trn_dat.shape}")
print(f"train amp  shape: {trn_amp.shape}")
print(f"test  data shape: {tst_dat.shape}")
print(f"test  amp  shape: {tst_amp.shape}")
print(f"val   data shape: {val_dat.shape}")
print(f"val   amp  shape: {val_amp.shape}")

## Visualising the data

Below we will make some kinematic plots of the events in the training sample.  Note however that these are not the physical distributions we would measure at the LHC!  In our training data each of these events is associated with an amplitude, which tells us the probability that the event will be produced in the gluon-gluon interaction.  So to get the physical distributions these events would need to be 'weighted' by their amplitude.  However, right now we just want to visualise our training dataset to see what preprocessing we should do.

In [ ]:
def get_init_pz( ev ):
    return ev[0][3] + ev[1][3]

def get_mass( fv ):
    msq = np.round( fv[0]**2 - fv[1]**2 - fv[2]**2 - fv[3]**2 , 5 )
    if msq>0:
        return np.sqrt( msq )
    elif msq<0:
        raise Exception( "mass squared is less than zero" )
    else:
        return 0

def get_pt( fv ):
    ptsq = np.round( fv[1]**2 + fv[2]**2 , 5 )
    if ptsq>0:
        return np.sqrt( ptsq )
    elif ptsq<0:
        raise Exception( "$p_T$ squared is less than zero" )
    else:
        return 0

def get_met( ev ):
    return np.abs( np.sum( [ fv[1]+fv[2] for fv in ev ] ) )

Plotting the initial $p_z$ of the events in the training sample.

In [ ]:
trn_dat_init_pz = []
for ev in trn_dat:
    trn_dat_init_pz.append( get_init_pz( ev ) )

In [ ]:
fig, axs = plt.subplots( 1, 1, figsize=(7,5) )

axs.hist( trn_dat_init_pz, histtype='stepfilled', fill=None )

axs.set_xlabel( "initial $p_z$ (GeV)")
axs.set_ylabel( "number of events")

fig.tight_layout()

In [ ]:
fig, axs = plt.subplots( 1, 1, figsize=(7,5) )

bins=np.logspace(-9, -3, 21)
axs.hist( trn_amp, histtype='stepfilled', fill=None, bins=bins )

axs.set_yscale( 'log' )
axs.set_xscale( 'log' )

axs.set_xlabel( "train amplitudes" )
axs.set_ylabel( "number of events" )

fig.tight_layout()

Plotting the leading photon $p_T$.

In [ ]:
trn_dat_leading_photon_pt = []
for ev in trn_dat:
    trn_dat_leading_photon_pt.append( get_pt( ev[2] ) )

In [ ]:
fig, axs = plt.subplots( 1, 1, figsize=(7,5) )

axs.hist( trn_dat_leading_photon_pt, histtype='stepfilled', fill=None )

axs.set_yscale( 'log' )

axs.set_xlabel( "leading photon $p_T$ (GeV)")
axs.set_ylabel( "number of events" )

fig.tight_layout()

## Add systematic noise to the data

We now add a systematic noise component to the amplitudes. For instance, relative Gaussian noise sampled as $\epsilon\sim \mathcal{N}(0, \sigma A)$

In [ ]:
def add_relative_gaussian_noise(amp, relative_std):
  """
  Adds relative Gaussian noise to an array of amplitudes.
  """
  std = amp * relative_std
  noise = np.random.normal(loc=0, scale=std, size=amp.shape)
  return amp + noise

sigma_noise = 0.1
trn_amp_amp = add_relative_gaussian_noise(trn_amp, sigma_noise)
val_amp_amp = add_relative_gaussian_noise(val_amp, sigma_noise)
tst_amp_amp = add_relative_gaussian_noise(tst_amp, sigma_noise)

## Preprocessing the data

We will be using a dense network, so the data needs to be in vector format.  We will collapse the data for each event to a single vector of dimension $5\times4=20$.  The fact that the data is ordered here is important.  To predict the amplitude given the kinematics, the network needs to know which entries correspond to which particles in the process.

In [ ]:
def get_pt(fv):
    """ returns p_T of given four vector """
    ptsq = np.round(fv[:, 1]**2 + fv[:, 2]**2, 5)
    return np.sqrt(ptsq)

In [ ]:
# index 2 is leading photon, not gluon (which is 4)
trn_dat_gluon_pt = get_pt(trn_dat[:, 4])



In [ ]:
nev = trn_dat.shape[0]
trn_datf = np.reshape(trn_dat, (nev,-1))
val_datf = np.reshape(val_dat, (nev,-1))
tst_datf = np.reshape(tst_dat, (nev,-1))

In [ ]:
trn_datf.shape

**Exercise:** After having looked at the data, apply the preprocessing step you consider helpful to improve the training dynamics.
Hint: neural networks like numbers of $O(1)$

In [ ]:
# Your code here...

## Datasets and dataloaders

In [ ]:
class amp_dataset(Dataset):

    def __init__(self, data, amp):
        self.data = data
        self.amp = amp

    def __len__(self):
        return len(self.amp)

    def __getitem__(self, idx):
        return self.data[idx], self.amp[idx]

In [ ]:
trn_dataset = amp_dataset(trn_datfp, trn_amplp.unsqueeze(-1))
val_dataset = amp_dataset(val_datfp, val_amplp.unsqueeze(-1))
tst_dataset = amp_dataset(tst_datfp, tst_amplp.unsqueeze(-1))

In [ ]:
trn_dataloader = DataLoader(trn_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
tst_dataloader = DataLoader(tst_dataset, batch_size=64, shuffle=True)

## Building a Bayesian layer

First let's look at the source code for a **basic linear layer** in pytorch:

(https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear)

```
class Linear(Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
```

Objects of this class apply a linear transformation to the incoming data: $y = xA^T + b$.

The input arguments are required to initialise the layer, so, in the \_\_init()\_\_ function.  We have:
- in_features: size of each input sample
- out_features: size of each output sample
- bias: If set to ``False``, the layer will not learn an additive bias.  Default: ``True``

The shapes are:
- Input: $(*, H_{in})$ where $*$ means any number of dimensions including none and $H_{in} = \text{in_features}$.
- Output: $(*, H_{out})$ where all but the last dimension are the same shape as the input and $H_{out} = \text{out_features}$.

The layer has attributes:
- weight: the learnable weights of the module of shape $(\text{out_features}, \text{in_features})$. The values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$, where $k = \frac{1}{\text{in_features}}$
- bias:   the learnable bias of the module of shape $(\text{out_features})$.  If `bias` is ``True``, the values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k = \frac{1}{\text{in_features}}$.

Examples::

    >>> m = nn.Linear(20, 30)
    
    >>> input = torch.randn(128, 20)
    
    >>> output = m(input)
    
    >>> print(output.size())
    
    torch.Size([128, 30])

From the lecture, we know that in a Bayesian network the weights are replaced by Gaussian distributions, and on a forward pass we get the output by sampling from that distribution.

So the biases are the same as in the linear layer.  But not each weight is a Gaussian distibution and so needs a mean and a variance.  The bias and the mean and variance of the weight distribution will be learnable.  In practice it's easier to work with the log of the variance as it is more stable when optimising the network.

We need to be able to sample from the Gaussian weight distributions, and compute derivatives of the output in order to update the network parameters. To do this we use something called the 're-parameterisation trick'. It involves sampling random noise from a unit normal distribution, and then transforming that number using the mean and variance of the weight distribution in this way:
\begin{equation}
w = \mu + \sigma\times r
\end{equation}

where $r$ is a random number sampled from a unit normal distribution (Gaussian with mean and variance equal to one), $\mu$ is the mean of the weight distribution, and $\sigma$ is the standard deviation. In this way we separate the stochastic part of the function from the parameters defining the distribution. And so if we take any differentiable function of $x$ (e.g. an activation function), we can compute derivatives of that function with respect to the mean and variance of the weight distribution.

In the `forward` method we then need to implement this reparameterisation trick for the weights, and then apply the same linear transformation as in the standard linear layer.

On each forward pass we need to generate a set of random numbers with the same shape as our means and variances.  Choosing a set of random numbers for the sampling is equivalent to 'sampling' a new neural network from the Bayesian neural network.  And sometimes at the end of the analysis, we will want to keep the same network for testing.  So we don't always want to re-sample the random numbers on each forward pass.  To control this we define a function called `reset_random` which allows to easily sample a new state.

We also need a `reset_parameters` function to reset the parameters in the network.  This is standard for layers in pytorch.  We use a slightly different function to do this than is used in the pytorch linear layer, as can be seen below.

From the lecture, you know that the weight distributions require a prior.  The simplest choice for this prior is just a Gaussian distribution with mean zero and variance of one.  Results are typically not too sensitive to this prior, as long as the values are within reasonable limits.  For example, $\mathcal{O}(1)$ means and standard deviations.  Going beyond $\mathcal{O}(1)$ numbers just leads to numerical instabilities in the training.  The Bayesian loss function contains a term coming from the KL divergence between the weight distributions in the network and their priors.  So the layers should have some funcitonality to return these values.

We also introduce the `map` option, which sets the weights to the learned mean without any sampling.

In [ ]:
import math

class VBLinear(nn.Module):
    def __init__(self, in_features, out_features, prior_prec=1.0, _map=False, std_init=-9):
        super(VBLinear, self).__init__()
        self.n_in = in_features
        self.n_out = out_features
        self.map = _map
        self.prior_prec = prior_prec
        self.random = None
        self.bias = nn.Parameter(torch.Tensor(out_features))
        self.mu_w = nn.Parameter(torch.Tensor(out_features, in_features))
        self.logsig2_w = nn.Parameter(torch.Tensor(out_features, in_features))
        self.std_init = std_init
        self.reset_parameters()

    def enable_map(self):
        self.map = True

    def disenable_map(self):
        self.map = False

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.mu_w.size(1))
        self.mu_w.data.normal_(0, stdv)
        self.logsig2_w.data.zero_().normal_(self.std_init, 0.001)
        self.bias.data.zero_()

    def reset_random(self):
        self.random = None
        self.map = False

    def sample_random_state(self):
        return torch.randn_like(self.logsig2_w).detach().cpu().numpy()

    def import_random_state(self, state):
        self.random = torch.tensor(state, device=self.logsig2_w.device,
                                   dtype=self.logsig2_w.dtype)

    def KL(self):
        logsig2_w = self.logsig2_w.clamp(-30, 11)
        kl = 0.5 * (self.prior_prec * (self.mu_w.pow(2) + logsig2_w.exp())
                    - logsig2_w - 1 - math.log(self.prior_prec)).sum()
        return kl

    def forward(self, input):
        if self.training:
            # local reparameterization trick is more efficient and leads to
            # an estimate of the gradient with smaller variance.
            # https://arxiv.org/pdf/1506.02557.pdf
            mu_out = F.linear(input, self.mu_w, self.bias)
            logsig2_w = self.logsig2_w.clamp(-30, 11)
            s2_w = logsig2_w.exp()
            var_out = F.linear(input.pow(2), s2_w) + 1e-12 # Needed to avoid NaNs from gradient of sqrt in next line!
            return mu_out + var_out.sqrt() * torch.randn_like(mu_out)

        else:
            if self.map:
                return F.linear(input, self.mu_w, self.bias)

            logsig2_w = self.logsig2_w.clamp(-30, 11)
            if self.random is None:
                self.random = torch.randn_like(self.mu_w)
            s2_w = logsig2_w.exp()
            weight = self.mu_w + s2_w.sqrt() * self.random
            return F.linear(input, weight, self.bias) #+ 1e-8

    def __repr__(self):
        return f"{self.__class__.__name__} ({self.n_in}) -> ({self.n_out})"

You can also find this class in the GitHub repository and simply import it in your code.

Let's test the layers, define:

In [ ]:
tlr0 = VBLinear(10, 5)
tlr1 = VBLinear(5, 2)

Now some test data, a batch of 20 vectors with 10 elements each:

In [ ]:
x = torch.rand(20, 10)

Now pass the data first through layer0 then through layer1:

In [ ]:
tlr1(tlr0(x))

Note this has the correct shape:

In [ ]:
tlr1(tlr0(x)).shape

Also try running the same data through the layer multiple times, you get different results.  This is because of the sampling in the Bayesian layer.

In [ ]:
for i in range(5):
    print(tlr0(x)[0][0])

## The Bayesian loss function

From the lectures we know that there are two parts to the Bayesian loss function:
- The negative log Gaussian
- the KL from the network weights

The second comes from the layers, and the first is defined below.  This negative log Gaussian term acts on the two outputs from the Bayesian neural network:
- the mean
- the variance (which we parameterise as the log variance)

In [ ]:
def neg_log_gauss(outputs, targets):

    mu = outputs[:, 0]
    logsigma2 = outputs[:, 1]
    out = torch.pow(mu - targets, 2) / (2 * logsigma2.exp()) + 1./2. * logsigma2

    return torch.mean(out)

## Building the Bayesian neural network

We'll build a simple network with one input and one output layer, and two hidden layers.  We define the dimensions of these layers below.

In [ ]:
ipt_dim = 20
opt_dim = 1
hdn_dim = 50

We can now use the same building blocks we already implemented in the first session.

In [ ]:
class VBLinearBlock(nn.Module):
    """A simple linear -> relu -> dropout block."""
    def __init__(self, in_features, out_features, dropout_rate=0.1):
        super().__init__()
        self.linear = VBLinear(in_features, out_features)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        return self.dropout(self.relu(self.linear(x)))

class BayesianAmpNet(nn.Module):
    """
    A neural network for regression with a Gaussian likelihood.
    Outputs the mean and log variance of the Gaussian distribution.
    """
    def __init__(self, input_dim, hidden_dims, training_size, dropout_rate=0.0):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for hidden_dim in hidden_dims:
            layers.append(VBLinearBlock(prev_dim, hidden_dim, dropout_rate))
            prev_dim = hidden_dim

        self.net = nn.Sequential(*layers)

        # Output layers for mean and log variance
        self.mean_layer = VBLinear(prev_dim, 1)
        self.log_var_layer = VBLinear(prev_dim, 1)

        self.training_size = training_size
        self.vb_layers = []
        self.vb_layers.append(self.mean_layer)
        self.vb_layers.append(self.log_var_layer)
        for layer in self.net:
            if isinstance(layer, VBLinear):
                self.vb_layers.append(layer)

    # we need the KL from the bayesian layers to compute the loss function
    def KL(self):
        kl = 0
        for vb_layer in self.vb_layers:
            kl += vb_layer.KL()
        return kl / self.training_size

    # let's put the neg_log_gauss in the network class aswell since it is key to bayesian networks
    def neg_log_gauss(self, outputs, targets):
        mu = outputs[0].squeeze(-1)
        logsigma2 = outputs[1].squeeze(-1)
        out = torch.pow(mu - targets, 2) / (2 * logsigma2.exp()) + 1./2. * logsigma2
        return torch.mean(out)

    def reset_random(self):
        for layer in self.vb_layers:
                layer.reset_random()

    def forward(self, x):
        features = self.net(x)
        mean = self.mean_layer(features)
        log_var = self.log_var_layer(features)
        return mean, log_var

Check if we have a GPU.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Initialise the neural network and send to the GPU if we have one.  We can also print the model to see what layers it has.

In [ ]:
# Define your favorite neural network here ...
# then test it. In particular verify that each call
# gives different outputs, also with the same input.

## Optimising (training) the neural network

The Bayesian loss function has two terms which we have already definedl; the negative los Gaussian, and the KL divergence between the network and the network prior.  The latter comes from the KL divergence over the weights in the network.

Now we can write a training loop for a single epoch.

In [ ]:
def train_epoch(dataloader, model, optimizer, scheduler):

    size = len(dataloader.dataset)
    model.train()
    loss_tot, kl_tot, neg_log_tot = 0.0, 0.0, 0.0
    loss_during_opt, kl_during_opt, neg_log_during_opt = 0., 0., 0.

    for batch, (X, y) in enumerate(dataloader):

        # Define a training loop which gives predictions
        # calculates the nll, the kl, and updates the network

        # print the training loss every 100 updates
        if batch % 100 == 0:
            current = batch * len( X )
            print(f"current batch loss: {loss:>8f} KL: {kl:>8f} Neg-log {nl:>8f}  [{current:>5d}/{size:>5d}]")
    loss_live = loss_during_opt/len(dataloader)
    kl_live = kl_during_opt / len(dataloader)
    nl_live = neg_log_during_opt / len(dataloader)

    return loss_live, kl_live, nl_live

To monitor the performance of the network on the regression task we want to calculate the loss of both the training data and the validation data on the same network, so we have the following functions:

In [ ]:
def val_pass(dataloader, model):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    nls = 0.0
    kls = 0.0
    vls = 0.0
    mse = 0.0

    # we don't need gradients here since we only use the forward pass
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            nl = model.neg_log_gauss(pred, y.reshape(-1))
            kl = model.KL()
            vl = nl.item() + kl.item()
            mse += torch.mean((pred[0].squeeze(-1) - y.reshape(-1))**2)
            nls += nl
            kls += kl
            vls += vl

    nls /= num_batches
    kls /= num_batches
    vls /= num_batches
    mse /= num_batches
    print(f'Val loss: {vls:.4f}, NLL: {nls:.4f}, KL: {kls:.4f}, MSE {mse:>8}')

    return nls.cpu().numpy(), kls.cpu().numpy(), vls, mse.cpu().numpy()

def trn_pass(dataloader, model):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    nls = 0.0
    kls = 0.0
    tls = 0.0
    mse = 0.0

    # we don't need gradients here since we only use the forward pass
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            nl = model.neg_log_gauss(pred, y.reshape(-1))
            kl = model.KL()
            tl = nl.item() + kl.item()
            mse += torch.mean((pred[0].squeeze(-1) - y.reshape(-1))**2)
            nls += nl
            kls += kl
            tls += tl

    nls /= num_batches
    kls /= num_batches
    tls /= num_batches
    mse /= num_batches
    print( f"avg trn loss per batch: {tls:>8f} KL: {kls:>8f} Neg-log {nls:>8f} MSE {mse:>8}" )

    return nls.cpu().numpy(), kls.cpu().numpy(), tls, mse.cpu().numpy()

Now we can train the model!

In [ ]:
# a useful function to present things clearer
def seperator():
    print( "-----------------------------------------------" )

# Set a batch size
# define train, test, and validation dataset and dataloader
# you code here...


# re-initialise the model, the optimizer, and th scheduler
# finally train the network

In [ ]:
# track train and val losses
trn_nl_losses = []
trn_kl_losses = []
trn_losses = []
trn_mse_losses = []
val_nl_losses = []
val_kl_losses = []
val_losses = []
val_mse_losses = []
trn_nl_losses_live = []
trn_kl_losses_live = []
trn_losses_live = []

for t in range(epochs):
    seperator()
    print(f"Epoch {t+1}")
    seperator()

    loss_live, kl_live, nl_live = train_epoch(trn_dataloader, model_bnn, optimizer, scheduler)
    trn_nl_losses_live.append(nl_live)
    trn_kl_losses_live.append(kl_live)
    trn_losses_live.append(loss_live)
    seperator()

    trn_nl_loss, trn_kl_loss, trn_loss, trn_mse_loss = trn_pass(trn_dataloader, model_bnn)
    trn_nl_losses.append(trn_nl_loss)
    trn_kl_losses.append(trn_kl_loss)
    trn_losses.append(trn_loss)
    trn_mse_losses.append(trn_mse_loss)
    seperator()

    val_nl_loss, val_kl_loss, val_loss, val_mse_loss = val_pass(val_dataloader, model_bnn)
    val_nl_losses.append(val_nl_loss)
    val_kl_losses.append(val_kl_loss)
    val_losses.append(val_loss)
    val_mse_losses.append(val_mse_loss)
    seperator()
    print( "|" )

print("Done!")

## Plot the train and validation losses as a function of the epochs

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(18,5))

c1 = 'tab:red'
c2 = 'tab:green'

axs[0].plot(trn_losses, label="train", color=c1)
axs[0].plot(val_losses, label="val", color=c2)
axs[0].plot(trn_losses_live, label="train live", color=c1, ls='dashed')
axs[0].set_xlabel("epoch")
axs[0].set_ylabel("Total loss")
axs[0].set_ylim(-3, 1)
axs[0].legend()

axs[1].plot(trn_nl_losses, label="train", color=c1)
axs[1].plot(val_nl_losses, label="val", color=c2)
axs[1].plot(trn_nl_losses_live, label="train live", color=c1, ls='dashed')
axs[1].set_xlabel("epoch")
axs[1].set_ylabel("Neg-log-gauss loss")
axs[1].set_ylim(-3, 1)
axs[1].legend()

axs[2].plot(trn_kl_losses, label="train", color=c1)
axs[2].plot(val_kl_losses, label="val", color=c2)
axs[2].plot(trn_kl_losses_live, label="train live", color=c1, ls='dashed')
axs[2].set_yscale('log')
axs[2].set_xlabel("epoch")
axs[2].set_ylabel("KL loss")
axs[2].legend()


axs[3].plot(trn_mse_losses, label="train", color=c1)
axs[3].plot(val_mse_losses, label="val", color=c2)
axs[3].set_yscale('log')
axs[3].set_xlabel("epoch")
axs[3].set_ylabel("MSE Loss")
axs[3].legend()

fig.tight_layout()

We can see that both the train and validation losses are being reduced during training, the model is fitting well!

#Part 2:

#### Outline / tasks:
- Repulsive Ensembles
- Study the results
  - Evaluate accuracy of the model
  - Calculate uncertainties
  - Visualize predicted amplitudes w/ uncertainties
  - Plot pull distributions
  - Discuss calibration of predicted uncertainties
- Questions/Exercises

In [ ]:
# Using the code from the first session and the previous part
# now train a respulsive ensemble using a similar network

# Hint: you can reuse the StackedLinear layers
# and the StackedLinearBlock from the previous session

## Plot the train and validation losses as a function of the epochs

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(13,5))

c1 = 'tab:red'
c2 = 'tab:green'

axs[0].plot(trn_losses, label="train", color=c1)
axs[0].plot(val_losses, label="val", color=c2)
axs[0].set_xlabel("epoch")
axs[0].set_ylabel("Total loss")
axs[0].set_ylim(-3.5, 1)
axs[0].legend()

axs[1].plot(trn_nl_losses, label="train", color=c1)
axs[1].plot(val_nl_losses, label="val", color=c2)
axs[1].set_xlabel("epoch")
axs[1].set_ylabel("Neg-log-gauss loss")
axs[1].set_ylim(-3.5, 1)
axs[1].legend()

axs[2].plot(trn_mse_losses, label="train", color=c1)
axs[2].plot(val_mse_losses, label="val", color=c2)
axs[2].set_yscale('log')
axs[2].set_xlabel("epoch")
axs[2].set_ylabel("MSE Loss")
axs[2].legend()

fig.tight_layout()

## Study the results

We will now discuss how to analyse the outputs of the Bayesian network, and how this gives us the ability to estimate the error on our analysis. This last step is crucial to the application of any numerical technique in physics.

Now we want to get some visualisation of how well our amplitude regression has worked.

The simplest thing we can do is to pass our data through the neural network to get a predicted amplitude for each event, then histogram this and compare it to the histogram of the true amplitudes.

In [ ]:
# Define two fuctions for the extraction of the predictions
# and the learned uncertainties from the VI and RE
def get_bnn_predictions(model, dataloader, n_monte=30):
    """
    Gets predictions from BayesianAmpNet for a given number of Monte Carlo samples.
    """
    with torch.no_grad():
        amps_samples = []
        sigma2_samples = []

        # Your code here...

    return amps_samples, sigma2_samples

def get_repulsive_predictions(model, dataloader):
    """
    Gets predictions from the repulsive ensemble
    For repulsive ensembles, the predictions are the outputs of each ensemble member.
    """
    with torch.no_grad():
        amps_samples = []
        sigma2_samples = []

        # Your code here...

    return amps_samples, sigma2_samples

def get_predictions(model, dataloader, n_monte=30):
    """
    Gets predictions for a given number of Monte Carlo times for
    both variational inference and repulsive ensemble.

    Args:
        model: A torch nn.Module
        dataloader: DataLoader for the data.
        n_monte: Number of Monte Carlo samples (only used for var. inference).

    Returns:
        A tuple containing:
            - amps_samples: numpy array of predicted means.
            - sigma2_samples: numpy array of predicted variances.
    """

    # Your code here...

In [ ]:
tst_dataloader = DataLoader(tst_dataset, batch_size=64, shuffle=False)

# Get predictions from the VI BNN
bnn_amps, bnn_sigma2 = get_predictions(model_bnn, tst_dataloader, n_monte=10)
print(f"BayesianAmpNet predictions shape: mean={bnn_amps.shape}, variance={bnn_sigma2.shape}")

# Get predictions from the repulsive ensemble
repulsive_amps, repulsive_sigma2 = get_predictions(model_re, tst_dataloader) # n_monte is ignored
print(f"BayesianAmpNetV2 predictions shape: mean={repulsive_amps.shape}, variance={repulsive_sigma2.shape}")

print("Mean bnn sigma2: ", np.mean(bnn_sigma2))
print("Mean re sigma2: ", np.mean(repulsive_sigma2))

In [ ]:
# First evaluate the accuracy of our model,
# calculate negative log likelihoods

bnn_nll =

repulsive_nll =

print("Negative Log Likelihood (VI):", bnn_nll)
print("Negative Log Likelihood (Repulsive Ensemble):", repulsive_nll)

# Then, look at the predicted uncertainties,
# calculate total, systematic, and statistical variance
bnn_sigma2_syst =
bnn_sigma2_stat =
bnn_sigma2_tot =

bnn_sigma_syst =
bnn_sigma_stat =
bnn_sigma_tot =

re_sigma2_syst =
re_sigma2_stat =
re_sigma2_tot =

re_sigma_syst =
re_sigma_stat =
re_sigma_tot =

bnn_delta =
re_delta =

In [ ]:
# Histograms of uncertainties (Syst, Stat, Total)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
bins_unc = np.logspace(-3, 1, 50)
ax[0].hist(bnn_sigma_syst, bins=bins_unc, histtype='step', label='Systematic')
ax[0].hist(bnn_sigma_stat, bins=bins_unc, histtype='step', label='Statistical')
ax[0].hist(bnn_sigma_tot, bins=bins_unc, histtype='stepfilled', fill=True, alpha=0.3, label='Total')
ax[0].set_xscale('log')
ax[0].set_yscale('log')
ax[0].set_xlabel("$\sigma$")
ax[0].set_ylabel("Number of Events")
ax[0].set_title("BNN")
ax[0].legend()

ax[1].hist(re_sigma_syst, bins=bins_unc, histtype='step', label='Systematic')
ax[1].hist(re_sigma_stat, bins=bins_unc, histtype='step', label='Statistical')
ax[1].hist(re_sigma_tot, bins=bins_unc, histtype='stepfilled', fill=True, alpha=0.3, label='Total')
ax[1].set_xscale('log')
ax[1].set_yscale('log')
ax[1].set_xlabel("$\sigma$")
ax[1].set_ylabel("Number of Events")
ax[1].set_title("Repulsive Ensemble")
ax[1].legend()

In [ ]:
# Histograms of accuracies (Syst, Stat, Total)

fig, ax = plt.subplots(1, 2, figsize=(10, 5))
bins_delta = np.linspace(-3, 3, 50)
ax[0].hist(bnn_delta, bins=bins_delta, histtype='step', label='Rel. accuracy')
ax[0].set_yscale('log')
ax[0].set_xlabel("$\Delta$")
ax[0].set_ylabel("Number of Events")
ax[0].set_title("BNN")
ax[0].legend()

ax[1].hist(re_delta, bins=bins_delta, histtype='step', label='Rel. accuracy')
ax[1].set_yscale('log')
ax[1].set_xlabel("$\Delta$")
ax[1].set_ylabel("Number of Events")
ax[1].set_title("Repulsive Ensemble")
ax[1].legend()

In [ ]:
# Here we show an histogram of the predicted amplitudes with uncertainties

bins=np.linspace(-3, 3, 21)
values, true_test_bin = [], []
pred_reps =[]
n_rep = 100
for _ in range(n_rep):
    error = np.random.normal(loc=0.0, scale=bnn_sigma_tot, size=bnn_amps.shape[-1])
    data_rep = error + np.mean(bnn_amps, axis=0)
    pred_reps.append(data_rep)
pred_reps = np.stack(pred_reps, 0)
print("Shape of data w/ replicas: ", pred_reps.shape)

# Get histogram values
for i in range(len(pred_reps)):
    hist, bin_edges = np.histogram(pred_reps[i], bins=bins)
    # Normalize by the number of amplitudes
    hist = hist.astype('float')/hist.sum()
    values.append(hist)
bin_center = (bin_edges[:-1] + bin_edges[1:]) / 2
values = np.stack(values, axis=1)
mean_hist = np.mean(values, axis=1)
std_hist = np.std(values, axis=1)
true_hist, bin_edges = np.histogram(tst_ampl, bins=bins)
true_hist = true_hist.astype('float')/true_hist.sum()
pred_ens_hist, bin_edges = np.histogram(tst_ampl, bins=bins)
pred_ens_hist = pred_ens_hist.astype('float')/pred_ens_hist.sum()

In [ ]:
fig, ax = plt.subplots(3,1,figsize=[5,5], sharex=True, gridspec_kw={'height_ratios':(4,1,1), "hspace": 0.0})

# add ratio plot for test/true
ax[0].step(bin_edges[:-1], true_hist, where='post', label='True', color='C3')
ax[0].step(bin_edges[:-1], mean_hist, where='post', label="Mean", color='C1')
ax[0].fill_between(bin_edges[:-1], mean_hist+std_hist, mean_hist-std_hist, step="post", alpha=.3, color='C1')
ax[0].set_yscale( 'log' )
ax[0].legend()

ax[1].set_ylabel(r'$\frac{\text{Model}}{\text{Truth}}$')
ax[1].set_ylim(.7,1.3)
ax[1].get_xticklabels()

ratio = mean_hist/true_hist
ratio_isnan = np.isnan(ratio)
ratio[ratio_isnan] = 1.
delta = np.fabs(ratio - 1)*100

ax[1].fill_between(bin_edges[:-1], ((mean_hist-std_hist)/(true_hist)), ((mean_hist+std_hist)/(true_hist)), step='post',alpha=.3, color='tab:orange')
ax[1].step(bin_edges[:-1], (mean_hist/(true_hist + 1e-10)), color='tab:orange',  where='post')
ax[1].axhline(y=1., color='black', linestyle='-')
ax[1].axhline(y=1.2, color='tab:gray', linestyle='--')
ax[1].axhline(y=0.8, color='tab:gray', linestyle='--')

ax[2].set_ylabel(r"$\delta [\%]$")
ax[2].set_xlabel('truth amplitudes')
ax[2].set_ylim((0.05, 50))
ax[2].set_yscale("log")
ax[2].set_yticks([0.1, 1.0, 10.0])
ax[2].set_yticklabels([r"$0.1$", r"$1.0$", "$10.0$"])
ax[2].set_yticks([0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9,
                               2., 3., 4., 5., 6., 7., 8., 9., 20., 30., 40.,], minor=True)
ax[2].axhline(y=1.0, linewidth=0.5, linestyle="--", color="grey")
ax[2].errorbar(bin_center, delta, color='k', linewidth=1.0, fmt=".", capsize=2)

fig.tight_layout()

In [ ]:
# Define a function which calculates the pull values
# given the prediction, true values, and the predicted variances

def calculate_pulls(predictions, true_values, sigma_tot, sigma_stat, sigma_syst):
    # Your code here...

bnn_pull_total, bnn_pull_stat, bnn_pull_syst =
re_pull_total, re_pull_stat, re_pull_syst =

In [ ]:
from scipy.stats import norm

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot bnn pull distributions
bins = np.linspace(-5, 5, 50)
ax[0].hist(bnn_pull_total, bins=bins, density=True, alpha=0.6, label='BNN')
p = norm.pdf(bins, 0, 1)
ax[0].plot(bins, p, 'k--', linewidth=2, label='$\mathcal{N}(0,1)$')
ax[0].set_xlabel("$\\frac{A - \\bar{A}}{\\sigma_{total}}$")
ax[0].set_ylabel("Density")
ax[0].legend()

ax[1].hist(bnn_pull_syst, bins=bins, density=True, alpha=0.6, label='BNN')
ax[1].plot(bins, p, 'k--', linewidth=2, label='$\mathcal{N}(0,1)$')
ax[1].set_xlabel("$\\frac{A - \\bar{A}}{\\sigma_{syst}}$")
ax[1].set_ylabel("Density")
ax[1].legend()

ax[2].hist(bnn_pull_stat, bins=bins, density=True, alpha=0.6, label='BNN')
ax[2].plot(bins, p, 'k--', linewidth=2, label='$\mathcal{N}(0,1)$')
ax[2].set_xlabel("$\\frac{A - \\bar{A}}{\\sigma_{stat}}$")
ax[2].set_ylabel("Density")
ax[2].legend()

fig.tight_layout()
plt.show()


In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# Plot re pull distributions
bins = np.linspace(-5, 5, 50)
ax[0].hist(re_pull_total, bins=bins, density=True, alpha=0.6, label='RE')
p = norm.pdf(bins, 0, 1)
ax[0].plot(bins, p, 'k--', linewidth=2, label='$\mathcal{N}(0,1)$')
ax[0].set_xlabel("$\\frac{A - \\bar{A}}{\\sigma_{total}}$")
ax[0].set_ylabel("Density")
ax[0].legend()

ax[1].hist(re_pull_syst, bins=bins, density=True, alpha=0.6, label='RE')
ax[1].plot(bins, p, 'k--', linewidth=2, label='$\mathcal{N}(0,1)$')
ax[1].set_xlabel("$\\frac{A - \\bar{A}}{\\sigma_{syst}}$")
ax[1].set_ylabel("Density")
ax[1].legend()

ax[2].hist(re_pull_stat, bins=bins, density=True, alpha=0.6, label='RE')
ax[2].plot(bins, p, 'k--', linewidth=2, label='$\mathcal{N}(0,1)$')
ax[2].set_xlabel("$\\frac{A - \\bar{A}}{\\sigma_{stat}}$")
ax[2].set_ylabel("Density")
ax[2].legend()

fig.tight_layout()
plt.show()


## Questions&Exercises

  - Do you observe qualitative differences between the two approaches?
  - Has the neural network correctly learned the systematic noise? Hint: try to calculate the mean relative systematic ucertainty
  - Define your own noise and check whether the network can learn its distribution or not.
  - Investigate more carefully the repulsive ensemble (e.g. change the number of channels, the prior width, and the repulsive pre-factor)
  - Visualize predictions vs. a single kinematic variable: Choose one of the kinematic variables (e.g., leading photon $p_T$) and create a scatter plot showing the true amplitudes vs. this variable. This can help visualize how well the models are capturing the amplitude dependence on specific kinematics and how the uncertainties vary across the kinematic space.